In [1]:
! nvidia-smi
! python --version
! pip install optuna xgboost==1.6.2

/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Fri Feb 17 10:19:30 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.82.01    Driver Version: 470.82.01    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+--------------

In [2]:
! pip install -U kaleido

/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 14.4 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
# from pytorch_tabnet import TabNetRegressor
from sklearn.metrics import accuracy_score,mean_squared_error,mean_absolute_error,r2_score
import optuna as opt
import os
import joblib
import random

In [4]:
def make_save_cv_model(i,model_name,model,optim,mse_loss,trial_data,output_path="./"):

    ''' This function saves cross validation model in the corresponding directory ( if the path does not exist it creates the path for it'''


    if os.path.exists(os.path.join(output_path,f"trial_{i}_{model_name}_{optim}")):
        joblib.dump(model, os.path.join(output_path,f"trial_{i}_{model_name}_{optim}/{i}_model.z"))
        with open(os.path.join(output_path,f"trial_{i}_{model_name}_{optim}/losses.txt"),"w+") as file:file.write(f" mse_loss :: {str(mse_loss)}")
        joblib.dump(trial_data, os.path.join(output_path,f"trial_{i}_{model_name}_{optim}/{i}_trial_data.z"))
    else:
        os.mkdir(os.path.join(output_path,f"trial_{i}_{model_name}_{optim}"))
        joblib.dump(model, os.path.join(output_path,f"trial_{i}_{model_name}_{optim}/{i}_model.z"))
        with open(os.path.join(output_path,f"trial_{i}_{model_name}_{optim}/losses.txt"),"w+") as file:file.write(f" mse_loss :: {str(mse_loss)}")
        joblib.dump(trial_data, os.path.join(output_path,f"trial_{i}_{model_name}_{optim}/{i}_trial_data.z"))

In [5]:
def save_cv_model(i,model_name,model,optim,mse_loss,output_path="./"):

    ''' This function saves cross validation model in the corresponding directory ( if the path does not exist it creates the path for it'''


    if os.path.exists(os.path.join(output_path,f"{i}_{model_name}_{optim}")):
        joblib.dump(model, os.path.join(output_path,f"{i}_{model_name}_{optim}/{i}_model.z"))
        with open(os.path.join(output_path,f"{i}_{model_name}_{optim}/losses_{fold}.txt"),"w+") as file:file.write(f" mse_loss :: {str(mse_loss)}")
    else:
        os.mkdir(os.path.join(output_path,f"{i}_{model_name}_{optim}"))
        joblib.dump(model, os.path.join(output_path,f"{i}_{model_name}_{optim}/{i}_model.z"))
        with open(os.path.join(output_path,f"{i}_{model_name}_{optim}/losses_{fold}.txt"),"w+") as file:file.write(f" mse_loss :: {str(mse_loss)}")

In [6]:
def get_trial_data(trial) -> list:
  ''' This function takes the trial objects and returns the dictionary containing the trial details for plotting and comparing purposes '''
  trial_data = trial.get_trials()
  value_dict = {}
  for i in trial_data:
    print(i.params)
    value_dict[i.number] = {"params": i.params , "rmse": i.values}
    print(f"{i.number} : {i.values}")
  return value_dict

In [7]:
def save_optuna_plots(study,dirname):
    optim_hist = opt.visualization.plot_optimization_history(study)
    intermediate = opt.visualization.plot_intermediate_values(study)
    parallel = opt.visualization.plot_parallel_coordinate(study)
    plot_slice = opt.visualization.plot_slice(study)
    name_lists = ["optim_hist","intermediate","parallel", "plot_slice"]
    plot_lists = [optim_hist,intermediate,parallel, plot_slice]
    for name,plot in zip(name_lists,plot_lists):
        if os.path.exists(f"./{dirname}"):
            print("getting into if block")
            plot.write_image(f"./{dirname}/{name}.jpg")
        else:
            print("getting into else block")
            os.mkdir(f"./{dirname}")
            plot.write_image(f"./{dirname}/{name}.jpg")

In [8]:
def train_trial(fold_dict,fold,model_name,sc_df,tar_col,optim,optim_trial,k_folds,tar_cols="",verbose=1):

    ''' this function is used to train the model with parameters optimization using optuna and cross validation using stratified k_folds'''

    y = sc_df[tar_col]
    print(y.shape)
    x = sc_df.drop([tar_col],axis=1)
    print(x.shape)
    model_name = model_name 
    def objective(trial):
      train_index = fold_dict[fold]["train"]
      test_index = fold_dict[fold]["test"]
      clf = XGBRegressor(n_estimators=trial.suggest_categorical("xgb_est",[4500,5000]),
                         learning_rate=trial.suggest_categorical("xgb_lr",[0.01,3e-4,0.1]),
                         booster = trial.suggest_categorical("xgb_booster",["gbtree","gblinear","dart"]),
                         tree_method = trial.suggest_categorical("xgb_treemethod",["gpu_hist"]),
                         predictor = trial.suggest_categorical("xgp_predictor",["gpu_predictor"]))
      # print(f" train_index :: {train_index}")
      # print(f" test_index :: {test_index}")
      X_train,X_test = x.iloc[train_index,:], x.iloc[test_index,:]
      # print(X_train.shape, X_test.shape)
      X_train, X_test = X_train.to_numpy(dtype=np.float64), X_test.to_numpy(dtype=np.float64)
      Y_train, Y_test = y.iloc[train_index].to_numpy(dtype=np.float64), y.iloc[test_index].to_numpy(np.float64)
      # Y_train, Y_test = Y_train.to_numpy(dtype=np.float64), Y_test.to_numpy(dtype=np.float64)
      print(X_train.shape)
      print(Y_train.shape)
      print(X_test.shape)
      print(Y_test.shape)
      clf.fit(X_train, Y_train,
              eval_set=[(X_test, Y_test)],
              eval_metric=["rmse"])
      Y_pred = clf.predict(X_test)
      mse_error = mean_squared_error(Y_pred, Y_test, squared=False)
      return mse_error

    print(f"Starting optimization for fold : [{fold}/{k_folds}]")
    study = opt.create_study(direction='minimize')
    study.optimize(objective, n_trials=optim_trial)
    save_optuna_plots(study,"optuna_plots")
    best_params = study.best_params
    trial_data = get_trial_data(study)
    print(f" Best params for fold : [{fold}/{k_folds}]")
    print(best_params)
    train_index = fold_dict[fold]["train"]
    test_index = fold_dict[fold]["test"]
    X_train,X_test = x.iloc[train_index,:], x.iloc[test_index,:]
    # print(X_train.shape, X_test.shape)
    X_train, X_test = X_train.to_numpy(dtype=np.float64), X_test.to_numpy(dtype=np.float64)
    Y_train, Y_test = y.iloc[train_index], y.iloc[test_index]
    Y_train, Y_test = Y_train.to_numpy(dtype=np.float64), Y_test.to_numpy(dtype=np.float64)
    clf_model = XGBRegressor(**study.best_params)
    clf_model.fit(X_train,Y_train)
    Y_pred = clf_model.predict(X_test)
    error = {"mse_error" : mean_squared_error(Y_pred, Y_test, squared=False),
    "mae_error" : mean_absolute_error(Y_pred,Y_test),
    "rmse_error" : mean_squared_error(Y_pred, Y_test),
    "r2_score" : r2_score(Y_pred,Y_test) }
    # try:
    print("[++] Saving the model and parameters in corresponding directories")
    make_save_cv_model(fold,model_name,clf_model,optim,mse_loss=error,trial_data=trial_data)
    return trial_data,best_params

In [9]:
def train(fold_dict,fold,model_name,sc_df,tar_col,optim,k_folds,best_params,tar_cols="",verbose=1):

    ''' this function is used to train the model with parameters optimization using optuna and cross validation using stratified k_folds'''

    y = sc_df[tar_col]
    print(y.shape)
    x = sc_df.drop([tar_col],axis=1)
    print(x.shape)
    model_name = model_name 
    train_index = fold_dict[fold]["train"]
    test_index = fold_dict[fold]["test"]
    X_train,X_test = x.iloc[train_index,:], x.iloc[test_index,:]
    # print(X_train.shape, X_test.shape)
    X_train, X_test = X_train.to_numpy(dtype=np.float64), X_test.to_numpy(dtype=np.float64)
    Y_train, Y_test = y.iloc[train_index], y.iloc[test_index]
    Y_train, Y_test = Y_train.to_numpy(dtype=np.float64), Y_test.to_numpy(dtype=np.float64)
    clf_model = XGBRegressor(**best_params)
    clf_model.fit(X_train,Y_train,
                 eval_set=[(X_test, Y_test)],
                  eval_metric=["rmse"])
    Y_pred = clf_model.predict(X_test)
    error = {"mse_error" : mean_squared_error(Y_pred, Y_test, squared=False),
    "mae_error" : mean_absolute_error(Y_pred,Y_test),
    "rmse_error" : mean_squared_error(Y_pred, Y_test),
    "r2_score" : r2_score(Y_pred,Y_test) }
    # try:
    print("[++] Saving the model and parameters in corresponding directories")
    save_cv_model(fold,model_name,clf_model,optim,mse_loss=error)

In [10]:
use_df = pd.read_csv("../input/perov-scaled-data/scaled_trainable.csv")
tar_col = "JV_default_PCE_numeric"
model_name = "xg_boost_reg"
fold_dict = joblib.load("../input/perov-fold-data/fold_data_export.z")
optim = "no_optim"
k_folds = 20
trial_fold = random.choice([x for x in range(k_folds)])
num_trials = 10

trial_data,best_params = train_trial(fold_dict = fold_dict,
      fold = trial_fold,
      k_folds=k_folds,
      model_name=model_name,
      sc_df=use_df,
      tar_col=tar_col,
      optim = optim,
      optim_trial = num_trials)
for key,value in trial_data.items():
  print(f"{key}: {value['rmse']}")
print(f"[++] Ended the training process for fold {trial_fold}")


main_folds = [x for x in range(k_folds)]
for fold in main_folds:
    train(fold_dict = fold_dict,
          fold = fold,
          k_folds=k_folds,
          model_name=model_name,
          sc_df=use_df,
          tar_col=tar_col,
          optim = optim,
          best_params = best_params)
    print(f"[++] Ended the training process for fold {fold}")

[I 2023-02-17 10:20:01,380] A new study created in memory with name: no-name-cb1c04cf-3cd5-474d-b404-33ec448050e2


(46820,)
(46820, 103)
Starting optimization for fold : [18/20]
(44479, 103)
(44479,)
(2341, 103)
(2341,)


/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:797: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  UserWarning,


[0]	validation_0-rmse:21.28646
[1]	validation_0-rmse:21.28028
[2]	validation_0-rmse:21.27411
[3]	validation_0-rmse:21.26793
[4]	validation_0-rmse:21.26176
[5]	validation_0-rmse:21.25559
[6]	validation_0-rmse:21.24942
[7]	validation_0-rmse:21.24326
[8]	validation_0-rmse:21.23709
[9]	validation_0-rmse:21.23093
[10]	validation_0-rmse:21.22477
[11]	validation_0-rmse:21.21861
[12]	validation_0-rmse:21.21245
[13]	validation_0-rmse:21.20629
[14]	validation_0-rmse:21.20014
[15]	validation_0-rmse:21.19399
[16]	validation_0-rmse:21.18784
[17]	validation_0-rmse:21.18169
[18]	validation_0-rmse:21.17555
[19]	validation_0-rmse:21.16940
[20]	validation_0-rmse:21.16326
[21]	validation_0-rmse:21.15712
[22]	validation_0-rmse:21.15098
[23]	validation_0-rmse:21.14484
[24]	validation_0-rmse:21.13871
[25]	validation_0-rmse:21.13258
[26]	validation_0-rmse:21.12645
[27]	validation_0-rmse:21.12032
[28]	validation_0-rmse:21.11420
[29]	validation_0-rmse:21.10808
[30]	validation_0-rmse:21.10196
[31]	validation_0-

[I 2023-02-17 10:20:42,246] Trial 0 finished with value: 6.448355258776803 and parameters: {'xgb_est': 4500, 'xgb_lr': 0.0003, 'xgb_booster': 'gbtree', 'xgb_treemethod': 'gpu_hist', 'xgp_predictor': 'gpu_predictor'}. Best is trial 0 with value: 6.448355258776803.
/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:797: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  UserWarning,


...

[0]	validation_0-rmse:15.24630
[1]	validation_0-rmse:10.99302
[2]	validation_0-rmse:8.13274
[3]	validation_0-rmse:6.21349
[4]	validation_0-rmse:4.98905
[5]	validation_0-rmse:4.21494
[6]	validation_0-rmse:3.74270
[7]	validation_0-rmse:3.47259
[8]	validation_0-rmse:3.31765
[9]	validation_0-rmse:3.19826
[10]	validation_0-rmse:3.12832
[11]	validation_0-rmse:3.07581
[12]	validation_0-rmse:3.04381
[13]	validation_0-rmse:2.99893
[14]	validation_0-rmse:2.97453
[15]	validation_0-rmse:2.96880
[16]	validation_0-rmse:2.95618
[17]	validation_0-rmse:2.92056
[18]	validation_0-rmse:2.89989
[19]	validation_0-rmse:2.88995
[20]	validation_0-rmse:2.87037
[21]	validation_0-rmse:2.86372
[22]	validation_0-rmse:2.85765
[23]	validation_0-rmse:2.84055
[24]	validation_0-rmse:2.83536
[25]	validation_0-rmse:2.83104
[26]	validation_0-rmse:2.82156
[27]	validation_0-rmse:2.81395
[28]	validation_0-rmse:2.80356
[29]	validation_0-rmse:2.79817
[30]	validation_0-rmse:2.78257
[31]	validation_0-rmse:2.76065
[32]	validation_

/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:797: UserWarning:

`eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.



[0]	validation_0-rmse:15.15736
[1]	validation_0-rmse:10.90591
[2]	validation_0-rmse:8.04140
[3]	validation_0-rmse:6.13397
[4]	validation_0-rmse:4.91527
[5]	validation_0-rmse:4.14276
[6]	validation_0-rmse:3.68706
[7]	validation_0-rmse:3.41558
[8]	validation_0-rmse:3.25457
[9]	validation_0-rmse:3.15363
[10]	validation_0-rmse:3.08432
[11]	validation_0-rmse:3.04970
[12]	validation_0-rmse:2.99896
[13]	validation_0-rmse:2.97027
[14]	validation_0-rmse:2.95700
[15]	validation_0-rmse:2.94243
[16]	validation_0-rmse:2.91924
[17]	validation_0-rmse:2.89209
[18]	validation_0-rmse:2.87962
[19]	validation_0-rmse:2.86454
[20]	validation_0-rmse:2.85123
[21]	validation_0-rmse:2.84224
[22]	validation_0-rmse:2.82257
[23]	validation_0-rmse:2.80679
[24]	validation_0-rmse:2.80166
[25]	validation_0-rmse:2.79663
[26]	validation_0-rmse:2.77331
[27]	validation_0-rmse:2.76906
[28]	validation_0-rmse:2.75669
[29]	validation_0-rmse:2.75367
[30]	validation_0-rmse:2.74059
[31]	validation_0-rmse:2.73695
[32]	validation_

/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:797: UserWarning:

`eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.



[0]	validation_0-rmse:15.17703
[1]	validation_0-rmse:10.91236
[2]	validation_0-rmse:8.03612
[3]	validation_0-rmse:6.12800
[4]	validation_0-rmse:4.91631
[5]	validation_0-rmse:4.18053
[6]	validation_0-rmse:3.74768
[7]	validation_0-rmse:3.48475
[8]	validation_0-rmse:3.33758
[9]	validation_0-rmse:3.23016
[10]	validation_0-rmse:3.16049
[11]	validation_0-rmse:3.09101
[12]	validation_0-rmse:3.06161
[13]	validation_0-rmse:3.03419
[14]	validation_0-rmse:3.00052
[15]	validation_0-rmse:2.97900
[16]	validation_0-rmse:2.96881
[17]	validation_0-rmse:2.94714
[18]	validation_0-rmse:2.93725
[19]	validation_0-rmse:2.91930
[20]	validation_0-rmse:2.90712
[21]	validation_0-rmse:2.88589
[22]	validation_0-rmse:2.86603
[23]	validation_0-rmse:2.84332
[24]	validation_0-rmse:2.82410
[25]	validation_0-rmse:2.82038
[26]	validation_0-rmse:2.81099
[27]	validation_0-rmse:2.80136
[28]	validation_0-rmse:2.79060
[29]	validation_0-rmse:2.76652
[30]	validation_0-rmse:2.74665
[31]	validation_0-rmse:2.74246
[32]	validation_